In [ ]:
# must download when new kernel created
%pip install git+https://github.com/huggingface/transformers -qqq
%pip install bitsandbytes -qqq
%pip install huggingface_hub -qqq
%pip install peft -qqq
%pip install accelerate -qqq
%pip install trl -qqq
%pip install wandb -qqq
%pip install ipywidgets -qqq

%pip install torch==2.1.0 torchvision torchaudio 

In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('ongjjbrandon/mistral-pjf-v3/model-gaflufph:v0', type='model')
artifact_dir = artifact.download()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from datasets import load_dataset
from huggingface_hub import notebook_login
base_model, adapter, new_model = "mistralai/Mistral-7B-v0.1" , "./artifacts/model-gaflufph:v0/", "ogbrandt/naive-merge"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(base_model, device_map="cpu")
model = PeftModel.from_pretrained(model, adapter)
model = model.merge_and_unload()
model.save_pretrained("/home/ubuntu/remote-training/merged_models")

In [ ]:
# Clear the memory footprint
del model, trainer
torch.cuda.empty_cache()

# Reload the base model
base_model_reload = AutoModelForCausalLM.from_pretrained(
    base_model, low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.bfloat16,
    device_map= {"": 0})
model = PeftModel.from_pretrained(base_model_reload, new_model)
model = model.merge_and_unload()

# Reload tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
!nvidia-smi